In [1]:
import os
import getpass
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model

load_dotenv()
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

llm = init_chat_model("gpt-4o-mini", temperature=0.0)

In [ ]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
        You are a helpful assistant in extracting key data from a passage.
        Extract the desired information from the following passage.

        Only extract the properties mentioned in the 'Extract' function.

        Passage: {passage}
    """ 
)

class Extract(BaseModel):
    first_name: str = Field(description="The first name of the person.")
    last_name: str = Field(description="The last name of the person.")
    age: int = Field(description="The age of the person.")
    location: str = Field(description="The location of the person.")
    sentiment: str = Field(description="The sentiment expressed in the passage.")

# *** Define the structured output for the LLM
# *** This makes LLM to return a structured answer instead of free text.
structured_llm = llm.with_structured_output(Extract)

In [ ]:
def extract_keyData(psg: str) -> Extract:
    prompt = tagging_prompt.invoke({"passage": psg})
    response = structured_llm.invoke(prompt)
    return response

# Example usage
if __name__ == "__main__":
    psg = "I absolutely love this product! I'm 26 y/o. It's been a game-changer for my daily routine. The quality is top-notch and the customer service is outstanding. I've recommended it to all my friends and family - Sarah Johnson, USA."
    keyData = extract_keyData(psg)
    print(keyData)

first_name='Sarah' last_name='Johnson' age=26 location='USA' sentiment='positive'
